<a href="https://colab.research.google.com/github/pansplawik/eegParkinsonCognitiveDisorders/blob/master/thirdVersion.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install mne

Importowanie pakietów

In [2]:
import mne
import numpy as np
import matplotlib.pyplot as plt
import os
import pandas as pd
import csv

Ściezki plików

In [3]:
labels = pd.read_csv('/content/drive/MyDrive/dyplom/dataset.csv', sep=';')
labels['PD'] = labels['PD'].replace(2, 1)
path='/content/drive/MyDrive/dyplom/EEG'

Preprocesing danych

In [4]:
def load_epochs_from_dir(path, labels):
    epochs_array = []
    liczba=1
    filenames = sorted(os.listdir(path))
    for filename, label in zip(filenames, labels['PD']):
      print(filename)
      if filename.endswith('.edf'):
        filepath = os.path.join(path, filename)
        raw = mne.io.read_raw_edf(filepath, preload=True,verbose=False)
        raw.info.set_montage('standard_1020')
        raw.info['bads'].append('Oz')
        annotations = raw.annotations
        for i in range(len(annotations)):
            if annotations.description[i] == "break":
                annotations.description[i] = "BAD"
            elif annotations.description[i] == 'Break end':
                annotations.description[i] = "BAD"
            elif annotations.description[i] == 'Miesnie':
                annotations.description[i] = "BAD"
            elif annotations.description[i] == 'Break start':
                annotations.description[i] = "BAD"
            elif annotations.description[i] == 'Ruch':
                annotations.description[i] = "BAD"
        raw.set_annotations(annotations)
        raw.filter(1, 40, fir_design='firwin')
        events = mne.make_fixed_length_events(raw, duration=1.0)
        unique_events, unique_indices = np.unique(events[:, 0], return_index=True)
        events = events[unique_indices]
        tmin = 0
        tmax = 2
        baseline = (0, 2)
        reject = dict(
              eeg=80e-6,
              )
        epochs = mne.Epochs(raw, events, event_id=None, tmin=tmin, tmax=tmax, baseline=baseline, preload=True,reject_by_annotation='BAD',reject=reject,verbose=False)
        epochs_array.append([epochs, label])
        del raw
        del annotations
    return epochs_array


PSD

In [28]:
def psds(epochs_array, labels):
    data = []
    for epochs, label in zip(epochs_array, labels):
        if epochs is not None and len(epochs) > 0:
            epochs_data = epochs.get_data()
            epo_spectrum = mne.time_frequency.psd_array_multitaper(epochs_data, 512, fmin=1, fmax=4, bandwidth=4, adaptive=True, normalization='length', output='power', n_jobs=-1, max_iter=100, verbose=False)
            psds_epoch, freqs = epo_spectrum
            combined_psds = []
            for p in psds_epoch:
                p = 10 * np.log10(p)
                psds_mean = p.mean(0)
                psds_std = p.std(0)
                combined_psds.append(psds_mean)
            combined_psds = np.array(combined_psds)
            data.append([combined_psds, label])

            del psds_epoch
            del freqs
            del epo_spectrum
    return data


In [ ]:
data = load_epochs_from_dir('/content/drive/MyDrive/dyplom/EEG',labels)
df = pd.DataFrame(data, columns=['value', 'label'])
df.to_csv('data.csv', index=False)

In [29]:
psd = psds([epochs for epochs, label in data], [label for epochs, label in data])
df = pd.DataFrame(psd, columns=['psds', 'label'])
df.to_csv('psd.csv', index=False)

Tworzenie zestawów

In [30]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(df['psds'], df['label'], test_size=0.1, random_state=42)
X_train_2d = np.array([x[0] for x in X_train]).reshape(len(X_train), -1)
X_test_2d = np.array([x[0] for x in X_test]).reshape(len(X_test), -1)
y_train=np.array(y_train)
y_train=y_train.transpose()
y_test=np.array(y_test)
y_test=y_test.transpose()

In [ ]:
print(len(X_train_2d))
print(len(y_train))
a=1;
for p in X_train_2d:
  print(a)
  a=a+1
  print((p))

In [32]:
from mne.decoding import Vectorizer
from sklearn.svm import SVC
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import cross_val_score
# Tworzenie i trenowanie modelu SVM
clf_svm_pip = make_pipeline(Vectorizer(), StandardScaler(), SVC())
clf_svm_pip.fit(X_train_2d, y_train)

# Przewidywanie etykiet dla danych testowych
predictions_svm = clf_svm_pip.predict(X_test_2d)

# Raport klasyfikacji
report_svm = classification_report(y_test, predictions_svm, target_names=['0', '1'])
print('SVM Classification Report:\n{}'.format(report_svm))

# Dokładność modelu SVM
acc_svm = accuracy_score(y_test, predictions_svm)
print("Accuracy of SVM model: {}".format(acc_svm))

# Obliczanie precyzji, czułości i miary F1
precision_svm, recall_svm, fscore_svm, support_svm = precision_recall_fscore_support(y_test, predictions_svm, average='macro')
print('Precision: {0}, Recall: {1}, F1-score: {2}'.format(precision_svm, recall_svm, fscore_svm))

# Walidacja krzyżowa z k-fold
k = 5  # Liczba podziałów
scores = cross_val_score(clf_svm_pip, X_train_2d, y_train, cv=k, scoring='accuracy')


# Obliczanie średniej i odchylenia standardowego z poszczególnych iteracji
mean_accuracy = scores.mean()
std_accuracy = scores.std()

print('Cross-Validation Results ({}-fold)'.format(k))
print('Mean Accuracy:', mean_accuracy)
print('Standard Deviation:', std_accuracy)

SVM Classification Report:
              precision    recall  f1-score   support

           0       1.00      0.75      0.86         4
           1       0.88      1.00      0.93         7

    accuracy                           0.91        11
   macro avg       0.94      0.88      0.90        11
weighted avg       0.92      0.91      0.91        11

Accuracy of SVM model: 0.9090909090909091
Precision: 0.9375, Recall: 0.875, F1-score: 0.8952380952380952
Cross-Validation Results (5-fold)
Mean Accuracy: 0.5210526315789473
Standard Deviation: 0.1057881644328515


In [33]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import classification_report
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from mne.decoding import Vectorizer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
from sklearn.model_selection import cross_val_score
# Definiowanie modelu
clf = make_pipeline(
    StandardScaler(),
    Vectorizer(),
    RandomForestClassifier()
)
# Trenowanie modelu na wszystkich danych treningowych
clf.fit(X_train_2d, y_train)

# Przewidywanie etykiet dla danych testowych
y_pred = clf.predict(X_test_2d)

# Raport klasyfikacji
report = classification_report(y_test, y_pred, target_names=['0', '1'])
print('Random Forest Classification Report:\n{}'.format(report))

# Dokładność modelu
acc = accuracy_score(y_test, y_pred)
print("Accuracy of model: {}".format(acc))

# Walidacja krzyżowa z k-fold
k = 5  # Liczba podziałów
scores = cross_val_score(clf, X_train_2d, y_train, cv=k, scoring='accuracy')


# Obliczanie średniej i odchylenia standardowego z poszczególnych iteracji
mean_accuracy = scores.mean()
std_accuracy = scores.std()

print('Cross-Validation Results ({}-fold)'.format(k))
print('Mean Accuracy:', mean_accuracy)
print('Standard Deviation:', std_accuracy)

Random Forest Classification Report:
              precision    recall  f1-score   support

           0       0.60      0.75      0.67         4
           1       0.83      0.71      0.77         7

    accuracy                           0.73        11
   macro avg       0.72      0.73      0.72        11
weighted avg       0.75      0.73      0.73        11

Accuracy of model: 0.7272727272727273
Cross-Validation Results (5-fold)
Mean Accuracy: 0.5099415204678361
Standard Deviation: 0.057345734138625815


In [34]:
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import StandardScaler
from mne.decoding import Vectorizer
from sklearn.metrics import precision_recall_fscore_support, accuracy_score
clf = make_pipeline(
StandardScaler(),
Vectorizer(),
LogisticRegression() # Używamy regresji logistycznej z domyślnymi parametrami
)
clf.fit(X_train_2d, y_train)

#Przewidywanie etykiet dla danych testowych
y_pred = clf.predict(X_test_2d)
acc = accuracy_score(y_test, y_pred)

print("Accuracy score: {}".format(acc))
# Walidacja krzyżowa z k-fold
k = 5  # Liczba podziałów
scores = cross_val_score(clf, X_train_2d, y_train, cv=k, scoring='accuracy')


# Obliczanie średniej i odchylenia standardowego z poszczególnych iteracji
mean_accuracy = scores.mean()
std_accuracy = scores.std()

print('Cross-Validation Results ({}-fold)'.format(k))
print('Mean Accuracy:', mean_accuracy)
print('Standard Deviation:', std_accuracy)

Accuracy score: 0.5454545454545454
Cross-Validation Results (5-fold)
Mean Accuracy: 0.5426900584795321
Standard Deviation: 0.06992156808250351


In [ ]:
from joblib import dump, load
dump(clf, 'oficjalnyRandomForest.joblib')

['oficjalnyRandomForest.joblib']